In [1]:
import pandas as pd

In [3]:
class_table = pd.read_csv('timetable_classifier.csv')
num_table = pd.read_csv('timetable_numeric.csv')
print(class_table.head())
print(num_table.head())

   Average Time  Activation  Observations  Dimensions  Depth
0      0.007197        ReLU           100           1      1
1      0.008350  Leaky_ReLU           100           1      1
2      0.007012         ELU           100           1      1
3      0.011740       PReLU           100           1      1
4      0.006306        SELU           100           1      1
   Average Time  Activation Underlying Relationship  Observations  Dimensions  \
0      0.008449        ReLU                     lin           100           1   
1      0.013152  Leaky_ReLU                     lin           100           1   
2      0.012639         ELU                     lin           100           1   
3      0.011070       PReLU                     lin           100           1   
4      0.009000        SELU                     lin           100           1   

   Depth  
0      1  
1      1  
2      1  
3      1  
4      1  


In [11]:
#Just doing classification table comparison

class_relu = class_table[class_table['Activation'] == 'ReLU']
class_leaky = class_table[class_table['Activation'] == 'Leaky_ReLU']
class_elu = class_table[class_table['Activation'] == 'ELU']
class_prelu = class_table[class_table['Activation'] == 'PReLU']
class_selu = class_table[class_table['Activation'] == 'SELU']

In [18]:
merge_list = ['Observations', 'Dimensions', 'Depth']

merged = class_relu.merge(class_leaky, on=merge_list, suffixes=('', '_leaky')) \
    .merge(class_elu, on=merge_list, suffixes=('', '_elu')) \
    .merge(class_prelu, on=merge_list, suffixes=('', '_prelu')) \
    .merge(class_selu, on=merge_list, suffixes=('', '_selu'))
merged

,Average Time,Activation,Observations,Dimensions,Depth,Average Time_leaky,Activation_leaky,Average Time_elu,Activation_elu,Average Time_prelu,Activation_prelu,Average Time_selu,Activation_selu
0,0.007197,ReLU,100,1,1,0.008350,Leaky_ReLU,0.007012,ELU,0.011740,PReLU,0.006306,SELU
1,0.019578,ReLU,100,1,3,0.011421,Leaky_ReLU,0.023004,ELU,0.018092,PReLU,0.020364,SELU
2,0.029844,ReLU,100,1,10,0.038356,Leaky_ReLU,0.078889,ELU,0.051347,PReLU,0.037293,SELU
3,0.087037,ReLU,100,1,30,0.085530,Leaky_ReLU,0.075959,ELU,0.107915,PReLU,0.091794,SELU
4,0.009165,ReLU,100,2,1,0.006038,Leaky_ReLU,0.006431,ELU,0.007113,PReLU,0.007082,SELU
5,0.010000,ReLU,100,2,3,0.013009,Leaky_ReLU,0.013692,ELU,0.014638,PReLU,0.012205,SELU
6,0.033149,ReLU,100,2,10,0.037495,Leaky_ReLU,0.046424,ELU,0.048370,PReLU,0.028599,SELU
7,0.086415,ReLU,100,2,30,0.118762,Leaky_ReLU,0.083127,ELU,0.130215,PReLU,0.130920,SELU
8,0.007111,ReLU,100,5,1,0.007010,Leaky_ReLU,0.007285,ELU,0.007505,PReLU,0.007610,SELU
9,0.011878,ReLU,100,5,3,0.012298,Leaky_ReLU,0.012260,ELU,0.014483,PReLU,0.032309,SELU


In [23]:
# Step 1: Collect all relevant columns into long format
long_df = pd.melt(
    merged,
    id_vars=['Observations', 'Dimensions', 'Depth'],
    value_vars=[
        'Average Time', 'Average Time_leaky', 'Average Time_elu',
        'Average Time_prelu', 'Average Time_selu'
    ],
    var_name='Activation',
    value_name='Average_Time'
)

# Step 2: Clean the 'Activation' names
long_df['Activation'] = long_df['Activation'].str.replace('Average Time_', '', regex=False)
long_df['Activation'] = long_df['Activation'].replace({'Average Time': 'relu'})

# Step 3: Pivot so that each activation becomes a column
pivot_df = long_df.pivot_table(
    index=['Observations', 'Dimensions', 'Depth'],
    columns='Activation',
    values='Average_Time'
).reset_index()

# Optional: if you want activation names as simple column names (not MultiIndex)
pivot_df.columns.name = None
#TY chat
# Get list of activation columns (assuming they start after the 3rd column)
activation_cols = pivot_df.columns[3:]

# Find the column with the lowest average time for each row
pivot_df['Best Activation'] = pivot_df[activation_cols].idxmin(axis=1)


In [24]:
pivot_df

,Observations,Dimensions,Depth,elu,leaky,prelu,relu,selu,Best Activation
0,100,1,1,0.007012,0.008350,0.011740,0.007197,0.006306,selu
1,100,1,3,0.023004,0.011421,0.018092,0.019578,0.020364,leaky
2,100,1,10,0.078889,0.038356,0.051347,0.029844,0.037293,relu
3,100,1,30,0.075959,0.085530,0.107915,0.087037,0.091794,elu
4,100,2,1,0.006431,0.006038,0.007113,0.009165,0.007082,leaky
5,100,2,3,0.013692,0.013009,0.014638,0.010000,0.012205,relu
6,100,2,10,0.046424,0.037495,0.048370,0.033149,0.028599,selu
7,100,2,30,0.083127,0.118762,0.130215,0.086415,0.130920,elu
8,100,5,1,0.007285,0.007010,0.007505,0.007111,0.007610,leaky
9,100,5,3,0.012260,0.012298,0.014483,0.011878,0.032309,relu


In [26]:
#Now to save
pivot_df.to_csv('class_time_comparison.csv', index = False)